In [1]:
from models import build_model_bert4Rec
from dataloader import Bert4RecDataLoader

import polars as pl
from tqdm import tqdm
tqdm.pandas()
import gc
from collections import defaultdict
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from datetime import datetime
from annoy import AnnoyIndex

physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    print('Invalid device or cannot modify virtual devices once initialized.')

2022-12-01 08:13:37.689400: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-01 08:13:37.814419: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-01 08:13:37.865792: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-01 08:13:38.399531: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2022-12-01 08:13:39.162889: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-01 08:13:39.241382: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-01 08:13:39.241480: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
# df_mapping = pd.read_csv('../tfrecords/tfrecords_v0.5/df_mapping.csv')
NUM_ITEMS = 1_855_603+1

# dict_map, dict_map_inv = {}, {}
# for x in tqdm(df_mapping.to_dict('records')):
#     dict_map[x['aid_map']] = x['aid']
#     dict_map_inv[x['aid']] = x['aid_map']

dict_map_type = {
    'clicks' : 1,
    'carts' : 2,
    'orders' : 3
  }

print(f'NUM_ITEMS: {NUM_ITEMS}')

NUM_ITEMS: 1855604


In [3]:
list_paths = ['../tfrecords/tfrecords_v0.6/na_split=val/' + x for x in os.listdir('../tfrecords/tfrecords_v0.6/na_split=val')]
# 5,45, 1,09
dataloader = Bert4RecDataLoader(list_paths, 
                                num_items=NUM_ITEMS, 
                                seq_len=80, 
                                seq_len_target=None,
                                batch_size=32, 
                                mask_prob=0.0, 
                                reverse_prob=0.0, 
                                get_session=True,
                                is_val=True,
                                is_test=False,
                                shuffle=False).get_generator()
# Val
list_sessions_val, list_items_val, list_types_val = [], [], []
for batch in tqdm(dataloader):
    features, targets, session = batch
    seq_items, seq_type, seq_time, seq_recency = features
    target, type_target, idx_mask = targets
    idxs = idx_mask-1
    list_sessions_val = list_sessions_val + session.numpy().tolist()
    list_items_val += [[item-1 for item in seq[:idxs[i]] if item!=0] for i, seq in enumerate(seq_items[:, :, 0].numpy())]
    list_types_val += [[type_ for type_ in seq[:idxs[i]] if type_!=0] for i, seq in enumerate(seq_type[:, :, 0].numpy())]
    # break
print(len(list_sessions_val))
# 20110it [02:36, 128.29it/s]

# del dataloader, batch, features, targets, session
# gc.collect()

2022-11-30 20:36:58.946608: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-30 20:36:58.947625: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-30 20:36:58.947718: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-30 20:36:58.947758: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA

647081


# 1. Validation

## 1.1 Model based predictions 

In [4]:
class BERT4REC_CONFIG:
    seed = 42 
    num_items = NUM_ITEMS
    model_arch = 'bert4rec'
    path_tfrecords = '../tfrecords/tfrecords_v0.6/'
    restore_last_chekpoint = (True, 'model_bert4rec_complete_0.11/checkpoints/', 'ckpt-25')
    model_name = 'model_bert4rec_complete_0.10'
    checkpoint_filepath = f'../2_Models/'
    num_records_dataset = 10_000_000
    batch_size = 32
    tup_scheduler_grad_accum = (5, 10, 1_500_000) #(start_grad_accum, max_grad_accum, ramp_up_samples)
    seq_len = 40
    mask_prob = 0.4
    reverse_prob = 0.5
    emb_dim = 128
    trf_dim = 128
    num_heads = 4
    num_layers = 1
    ff_dim = trf_dim*4
    drop_rate = 0.1
    att_drop_rate = 0.1
    epochs = 3
    early_stopping = 5
    batch_num_printer_train = 500
    batch_num_printer_val = 250
    clipnorm = 1.0
    num_iters_save_checkpoint = 25_000
    scheduler_scaler = 128 
    warmup_steps = 10_000
    weight_decay = 1e-1
    log_wandb = True

tf.keras.backend.clear_session()
model = build_model_bert4Rec(num_items=NUM_ITEMS, model_cfg=BERT4REC_CONFIG)
ckpt = tf.train.Checkpoint(model=model)
ckpt.restore(tf.train.latest_checkpoint(f'../2_Models/model_bert4rec_complete_0.11/checkpoints'))
# _ = model(features, training=False)
# del _
# gc.collect()

def get_score_metric(y_true, y_pred, type_target, k=20):
    y_pred = list(set(y_pred))[:k]
    score = 0 
    if len(y_true)==0:
        return None
    if type_target=='clicks':
        num_targets = 1
        hits = len([x for x in y_pred if x==y_true[0]])
    else:
        num_targets = min(k, len(y_true))
        hits = len([x for x in y_pred if x in y_true])
    score = hits / num_targets
    return score

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3090 Ti, compute capability 8.6


2022-11-30 20:39:27.635783: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [16]:
list_paths_val = ['../tfrecords/tfrecords_v0.6/na_split=val/' + x for x in os.listdir('../tfrecords/tfrecords_v0.6/na_split=val')]
val_dataloader = Bert4RecDataLoader(list_paths_val, 
                                     num_items=NUM_ITEMS, 
                                     seq_len=40, 
                                     seq_len_target=50, 
                                     batch_size=32, 
                                     mask_prob=0.0, 
                                     reverse_prob=0.0, 
                                     is_val=True,
                                     get_session=True, 
                                     is_test=False,
                                     shuffle=False).get_generator()

num_candidates = 20
list_sessions, list_past_items, list_predictions, list_trues, list_types = [], [], [], [], []
for num_batch, batch in enumerate(tqdm(val_dataloader)):
    features, targets, session = batch
    seq_items, seq_type, seq_time, seq_recency = features
    target, type_target, idx_mask = targets
    idxs = idx_mask.numpy() - 1
    for type_ in ['clicks', 'carts', 'orders']:
        seq_type_new = [tf.concat([
                        seq_type[i, :ix],
                        tf.constant([[dict_map_type[type_]]], tf.int64),
                        seq_type[i, ix+1:]], axis=0)
                    for i, ix in enumerate(idxs)]
        features = (seq_items, tf.stack(seq_type_new, axis=0), seq_time, seq_recency)
        preds = model(features, training=False)
        preds = tf.gather(preds, indices=idxs, axis=1, batch_dims=1)
        topk_scores, topk_idxs = tf.math.top_k(preds, k=num_candidates)
        topk_idxs = np.asarray([[x-1 for x in topk_idxs.numpy()[i, :-1]] for i in range(topk_idxs.numpy().shape[0])])
        last_item = np.asarray([[x[idx-1, 0]-1] for x, idx in zip(seq_items.numpy(), idxs)])
        new_predictions = np.concatenate([last_item, topk_idxs], axis=1)
        labels = [list(set([_target-1 for _type, _target in zip(type_target.numpy()[i], target.numpy()[i]) if dict_map_type[type_]==_type and _target!=0])) for i in range(target.shape[0])]
        ###
        list_sessions.append(session.numpy())
        list_predictions.append(topk_idxs)
        list_types.append([type_ for _ in range(seq_items.shape[0])])
        list_trues = list_trues + labels
        list_past_items.append(seq_items.numpy()[:, :, 0])
    if num_batch==1000:
        break

df_val_model = pd.DataFrame({
    'session' : np.concatenate(list_sessions),
    'past_items' : np.concatenate(list_past_items).tolist(),
    'predictions' : np.concatenate(list_predictions).tolist(),
    'trues' : list_trues,
    'type' : np.concatenate(list_types)
})

df_val_model['qt_trues'] = df_val_model['trues'].apply(lambda x : len(x))

del preds, features
gc.collect()

1000it [02:04,  8.04it/s]


5726

## 1.1 Rule based predictions

In [6]:
arr_embedding = model.layers[1].weights[0].numpy()
index = AnnoyIndex(arr_embedding.shape[1], 'euclidean')

# for idx, aid in dict_map.items():
#     index.add_item(idx, arr_embedding[idx])

for idx in tqdm(range(1, NUM_ITEMS)):
    aid = idx - 1
    index.add_item(aid, arr_embedding[idx])
    
index.build(10)

100%|██████████| 1855603/1855603 [00:07<00:00, 248748.60it/s]


True

In [7]:
session_types = ['clicks', 'carts', 'orders']

# val_session_items = df_val.to_pandas().reset_index(drop=True).groupby('session')['aid'].apply(list)
# val_session_types = df_val.to_pandas().reset_index(drop=True).groupby('session')['type'].apply(list)
# val_sessions = val_session_items.index

num_items_to_consider = 20
num_candidates = 20
type_weight_multipliers = {1: 1, 2: 6, 3: 3}
list_labels, list_sessions, list_len_past_items = [], [], []
for session, seq_items, seq_types in tqdm(zip(list_sessions_val, list_items_val, list_types_val), total=len(list_items_val)):
    if len(seq_items) >= num_items_to_consider:
        # if we have enough aids (over equals num_items_to_consider) we don't need to look for candidates! we just use the old logic
        weights=np.logspace(0.1, 1, len(seq_items), base=2, endpoint=True)-1
        aids_temp=defaultdict(lambda: 0)
        for aid,w,t in zip(seq_items, weights, seq_types): 
            aids_temp[aid] += w * type_weight_multipliers[t]
            
        sorted_aids=[k for k, v in sorted(aids_temp.items(), key=lambda item: -item[1])]
        list_labels.append(sorted_aids[:num_candidates])
    else:
        if len(seq_items)==0:
            seq_items = [1]
        # list_labels.append([0])
        # here we don't have num_items_to_consider aids to output -- we will use embeddings to generate candidates!
        seq_items_new = list(dict.fromkeys(seq_items[::-1]))
        
        # let's grab the most recent aid
        most_recent_aid = seq_items_new[0]
        
        # and look for some neighbors!
        nns = [i-1 for i in index.get_nns_by_item(most_recent_aid, num_candidates+1, search_k=-1)[1:]]
                        
        list_labels.append((seq_items_new+nns)[:num_candidates])

    list_sessions.append(session)
    list_len_past_items.append(len(seq_items))

# del list_items_val, list_types_val
# gc.collect()

df_val_rule = pd.DataFrame({
    'session' : list_sessions,
    'prediction' : list_labels,
    'qt_past_items' : list_len_past_items
})

# 100%|██████████| 647081/647081 [00:22<00:00, 28792.13it/s]


100%|██████████| 647081/647081 [00:19<00:00, 33543.43it/s]


## 1.3 Metrics

In [8]:
from collections import Counter
def emsemble_predictions(qt_past_items, type, list_pred_model, list_pred_embs):
    tmp_preds = Counter()
    if qt_past_items >= 20:
        for x in list_pred_embs:
            tmp_preds[x] += 1
        if len(tmp_preds) < 20:
            for x in list_pred_model:
                tmp_preds[x] += 1
    else:
        if type in ['clicks', 'carts']:
            for x in list_pred_model:
                tmp_preds[x] += 1
            for x in list_pred_embs:
                tmp_preds[x] += 1
        else:
            for x in list_pred_embs:
                tmp_preds[x] += 1
            for x in list_pred_model:
                tmp_preds[x] += 1
    final_preds = [k for k, v in tmp_preds.most_common(20)]
    return final_preds

In [18]:
df_val_merge = df_val_rule.merge(df_val_model, how='inner', on='session')
# df_val_merge['predictions_final'] = df_val_merge.progress_apply(lambda x : list(dict.fromkeys(x['predictions'] + x['prediction'])), axis=1)
df_val_merge['predictions_final'] = df_val_merge.progress_apply(lambda x : emsemble_predictions(x['qt_past_items'], x['type'], x['predictions'], x['prediction']), axis=1)
df_val_merge['score'] = df_val_merge.progress_apply(lambda x: get_score_metric(x['trues'], x['predictions_final'], x['type']), axis=1)

display(df_val_merge.describe())
dict_scores = df_val_merge.groupby('type')['score'].mean().to_dict()
display(dict_scores)
kaggle_metric = 0.1*dict_scores['clicks'] + 0.3*dict_scores['carts'] + 0.6*dict_scores['orders']
print(f'Kaggle Metric: {kaggle_metric:.4f}')

# # Model
# {'carts': 0.36967699795805764,
#  'clicks': 0.3317656031524088,
#  'orders': 0.49884798425002636}
# Kaggle Metric: 0.4434

# rule + embeddings
# {'carts': 0.33723911424885844,
#  'clicks': 0.2439938985636202,
#  'orders': 0.5639535939043883}
# Kaggle Metric: 0.4639

# final - ensemble
# {'carts': 0.39870859533618364,
#  'clicks': 0.34835388331002926,
#  'orders': 0.5649640172185767}
# Kaggle Metric: 0.4934

100%|██████████| 96096/96096 [00:00<00:00, 177994.70it/s]


,session,qt_past_items,qt_trues,score
count,9.609600e+04,96096.000000,96096.000000,41133.000000
mean,6.513860e+06,7.543300,1.763268,0.369885
std,3.737217e+06,13.521467,4.543446,0.471300
min,2.220000e+02,1.000000,0.000000,0.000000
25%,3.292913e+06,1.000000,0.000000,0.000000
50%,6.516554e+06,2.000000,0.000000,0.000000
75%,9.797466e+06,7.000000,1.000000,1.000000
max,1.289960e+07,78.000000,50.000000,1.000000


{'carts': 0.39558536656187565,
 'clicks': 0.3451707255039105,
 'orders': 0.5659949689258904}

Kaggle Metric: 0.4928


# Test

In [ ]:
del df_val_merge, df_val_model, df_val_rule, list_sessions_val, list_items_val, list_types_val
gc.collect()

## 2.1 Rule based

In [53]:
df_test = pl.read_parquet(
    '../0_Data/data_optimized/test.parquet'
)

In [10]:
session_types = ['clicks', 'carts', 'orders']

test_session_items = df_test.to_pandas().reset_index(drop=True).groupby('session')['aid'].apply(list)
test_session_types = df_test.to_pandas().reset_index(drop=True).groupby('session')['type'].apply(list)
test_sessions = test_session_items.index

num_items_to_consider = 20
type_weight_multipliers = {0: 1, 1: 6, 2: 3}
list_labels, list_sessions = [], []
for session, seq_items, seq_types in tqdm(zip(test_sessions, test_session_items, test_session_types), total=len(test_session_items)):
    if len(seq_items) >= num_items_to_consider:
        # if we have enough aids (over equals 20) we don't need to look for candidates! we just use the old logic
        weights=np.logspace(0.1, 1, len(seq_items), base=2, endpoint=True)-1
        aids_temp=defaultdict(lambda: 0)
        for aid,w,t in zip(seq_items, weights, seq_types): 
            if dict_map_inv[aid+1] == 0:
                aids_temp[aid] = 0
            else:  
                aids_temp[aid] += w * type_weight_multipliers[t]
            
        sorted_aids=[k for k, v in sorted(aids_temp.items(), key=lambda item: -item[1])]
        list_labels.append(sorted_aids[:num_items_to_consider])
    else:
        list_labels.append([0])
    list_sessions.append(session)

# del test_session_items, test_session_types
# gc.collect()

df_test_rule = pd.DataFrame({
    'session' : list_sessions,
    'prediction' : list_labels,
})

100%|██████████| 1671803/1671803 [00:02<00:00, 782559.23it/s] 


## 2.2 Model based predictions

In [55]:
list_paths_test = ['../tfrecords/tfrecords_v0.5/na_split=test/' + x for x in os.listdir('../tfrecords/tfrecords_v0.5/na_split=test')]
test_dataloader = Bert4RecDataLoader(list_paths_test, 
                                     num_items=NUM_ITEMS, 
                                     seq_len=30,  
                                     batch_size=54, 
                                     mask_prob=0.0, 
                                     reverse_prob=0.0,  
                                     is_val=False,
                                     is_test=True,
                                     get_session=True,
                                     shuffle=False).get_generator()

list_predictions, list_sessions, list_types, list_scores = [], [], [], []
for num_batch, batch in enumerate(tqdm(test_dataloader)):
    features, idxs, session = batch
    seq_items, seq_type, seq_time, seq_recency = features
    idxs = idxs.numpy() - 1
    ###
    for type_ in ['clicks', 'carts', 'orders']:
        seq_type_new = [tf.concat([
                        seq_type[i, :ix],
                        tf.constant([[dict_map_type[type_]]], tf.int64),
                        seq_type[i, ix+1:]], axis=0)
                    for i, ix in enumerate(idxs)]
        features = (seq_items, tf.stack(seq_type_new, axis=0), seq_time, seq_recency)
        preds = model(features, training=False)
        preds = tf.gather(preds, indices=idxs, axis=1, batch_dims=1)
        topk_scores, topk_idxs = tf.math.top_k(preds, k=20)
        topk_idxs = np.asarray([[dict_map[x]-1 for x in topk_idxs.numpy()[i, :]] for i in range(topk_idxs.numpy().shape[0])])
        last_item = np.asarray([[dict_map[x[idx-1, 0]]-1] for x, idx in zip(seq_items.numpy(), idxs)])
        new_predictions = np.concatenate([last_item, topk_idxs], axis=1)
        list_predictions.append(new_predictions)
        list_types.append([type_ for _ in range(seq_items.shape[0])])
        list_sessions.append(session.numpy())

# 41796it [1:22:29,  8.44it/s]


30960it [1:15:19,  6.85it/s]


In [15]:
name_submission = f"submission_{datetime.now().__str__().split('.')[0].replace(' ', '_').replace('-', '_').replace(':', '_')}"

df_inference = pd.DataFrame({
    'session' : np.concatenate(list_sessions),
    'predictions' : np.concatenate(list_predictions).tolist(),
    'type' : np.concatenate(list_types)
})

df_inference = df_inference.merge(df_test_rule, how='inner', on='session')
df_inference['predictions_final'] = df_inference.progress_apply(lambda x : x['predictions'] if len(x['prediction'])==1 else x['prediction'], axis=1)

df_inference['session_type'] = df_inference['session'].astype(str) + '_' + df_inference['type']
df_inference['labels'] = df_inference['predictions_final'].apply(lambda x : ' '.join([str(y) for y in x]))
df_inference[['session_type', 'labels']].to_csv(f'../3_Submissions/{name_submission}.csv', index=False)

print(df_inference.shape)
display(
    df_inference
)

import gzip
with open(f'../3_Submissions/{name_submission}.csv', 'rb') as f_in, gzip.open(f'../3_Submissions/{name_submission}.csv.gz', 'wb') as f_out:
    f_out.writelines(f_in)

(5015409, 7)


,session,predictions,type,prediction,predictions_final,session_type,labels
0,14309103,"[1048964, 539417, 744067, 1182248, 654388, 813...",clicks,[0],"[1048964, 539417, 744067, 1182248, 654388, 813...",14309103_clicks,1048964 539417 744067 1182248 654388 813656 81...
1,14309103,"[1048964, 744067, 539417, 1182248, 813816, 813...",carts,[0],"[1048964, 744067, 539417, 1182248, 813816, 813...",14309103_carts,1048964 744067 539417 1182248 813816 813656 65...
2,14309103,"[1048964, 744067, 539417, 1182248, 813816, 813...",orders,[0],"[1048964, 744067, 539417, 1182248, 813816, 813...",14309103_orders,1048964 744067 539417 1182248 813816 813656 16...
3,13784979,"[393449, 831848, 228982, 390399, 56279, 219222...",clicks,[0],"[393449, 831848, 228982, 390399, 56279, 219222...",13784979_clicks,393449 831848 228982 390399 56279 219222 18429...
4,13784979,"[393449, 831848, 228982, 219222, 748414, 18429...",carts,[0],"[393449, 831848, 228982, 219222, 748414, 18429...",13784979_carts,393449 831848 228982 219222 748414 1842909 891...
...,...,...,...,...,...,...,...
5015404,13408102,"[1358904, 396974, 898986, 1485262, 74336, 7555...",carts,[0],"[1358904, 396974, 898986, 1485262, 74336, 7555...",13408102_carts,1358904 396974 898986 1485262 74336 755541 132...
5015405,13408102,"[1358904, 396974, 898986, 1485262, 755541, 541...",orders,[0],"[1358904, 396974, 898986, 1485262, 755541, 541...",13408102_orders,1358904 396974 898986 1485262 755541 541728 62...
5015406,13857463,"[1838234, 276012, 32808, 1636724, 654388, 1027...",clicks,[0],"[1838234, 276012, 32808, 1636724, 654388, 1027...",13857463_clicks,1838234 276012 32808 1636724 654388 102778 730...
5015407,13857463,"[1838234, 276012, 32808, 774027, 1733659, 1115...",carts,[0],"[1838234, 276012, 32808, 774027, 1733659, 1115...",13857463_carts,1838234 276012 32808 774027 1733659 1115883 10...
